In [1]:
import os
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# https://pypi.org/project/category-encoders/
# https://contrib.scikit-learn.org/category_encoders/

# pip install category_encoders
import category_encoders as ce

In [3]:
# Here some default values that we're going to use in case we cannot calculate averages for precise subgroups

avglifetime = 850 # days
avghiretime = 75 # days

In [4]:
# instantiate labelencoder object

le = LabelEncoder()
lm = LinearRegression()

%matplotlib inline

In [5]:
calenda = pd.date_range(start='2021-06-30', periods=50, freq='M').to_frame()
today = pd.Timestamp('2021-09-30')

Based on the calculation of 4 data sets:
* current employees
* current open vacancies
* historical data on the time of employment of employees (in days) per features
* historical data on time to hire (in days) per features

In [6]:
# Creating an example array with features of current open vacancies
# Features: country, subfamily (occupation in the workplace), search department
# WorkStartDate - in this case, the date of opening the search for a position

d = {'Position ID': ['POS041296', 'POS027619', 'POS023292', 'POS018157', 'POS009096'],
     'Country': ['GB', 'GB', 'GB', 'GB', 'GB'],
     'Subfamily': ['SFA000065', 'SFA000096', 'SFA000036', 'SFA000086','SFA000033'],
     'Department': ['FT', 'RnD', 'IT', 'RnD','IT'],
     'WorkStartDate': ['2021-05-17', '2021-08-11', '2020-12-12', '2021-04-18','2021-04-20']}
     
df_vac = pd.DataFrame(data=d)
df_vac.head()

,Position ID,Country,Subfamily,Department,WorkStartDate
0,POS041296,GB,SFA000065,FT,2021-05-17
1,POS027619,GB,SFA000096,RnD,2021-08-11
2,POS023292,GB,SFA000036,IT,2020-12-12
3,POS018157,GB,SFA000086,RnD,2021-04-18
4,POS009096,GB,SFA000033,IT,2021-04-20


In [7]:
# Creating an example array with the characteristics of current employees (employes)
# Features: country, subfamily (occupation in the workplace), search department
# WorkStartDate - in this case, the date the employee started working in the organization

e = {'UserID': ['11391', '7642', '9018', '1359', '9552'],
     'Position ID': ['POS012903', 'POS040759', 'POS044017', 'POS027569', 'POS040784'],
     'Country': ['FR', 'GB', 'GB', 'GB','GB'],
     'Subfamily': ['SFA000065', 'SFA000096', 'SFA000036', 'SFA000086','SFA000033'],
     'Department': ['FT', 'RnD', 'IT', 'RnD','IT'],
     'WorkStartDate': ['2019-05-17', '2019-08-11', '2020-12-12', '2018-04-18','2021-04-20']}

df_empl = pd.DataFrame(data=e)
df_empl.head()

,UserID,Position ID,Country,Subfamily,Department,WorkStartDate
0,11391,POS012903,FR,SFA000065,FT,2019-05-17
1,7642,POS040759,GB,SFA000096,RnD,2019-08-11
2,9018,POS044017,GB,SFA000036,IT,2020-12-12
3,1359,POS027569,GB,SFA000086,RnD,2018-04-18
4,9552,POS040784,GB,SFA000033,IT,2021-04-20


In [8]:
# Create an example array with historical recruitment data (vacancy closing rates)
# Features: country, subfamily (occupation in the workplace)
# Y - the number of days from opening a position in the search to hiring an employee

vh = {'Country': ['FR', 'GB', 'GB', 'GB','GB'],
     'Subfamily': ['SFA000065', 'SFA000096', 'SFA000036', 'SFA000086','SFA000033'],
     'Y': [300, 200, 100, 400, 356]}

df_vac_hist = pd.DataFrame(data=vh)
df_vac_hist.head()

,Country,Subfamily,Y
0,FR,SFA000065,300
1,GB,SFA000096,200
2,GB,SFA000036,100
3,GB,SFA000086,400
4,GB,SFA000033,356


In [9]:
# Creating an example array with historical data on the lifetime of an employee
# (number of days between hiring and firing)

# Features: country, subfamily (occupation in the workplace)
# Y - the number of days from opening a position in the search to hiring an employee

eh = {'Country': ['FR', 'GB', 'GB', 'GB', 'GB'],
     'Subfamily': ['SFA000065', 'SFA000096', 'SFA000036', 'SFA000033','SFA000033'],
     'Y': [3000, 2000, 1000, 500, 1000]
     }

df_empl_hist = pd.DataFrame(data=eh)
df_empl_hist.head()

,Country,Subfamily,Y
0,FR,SFA000065,3000
1,GB,SFA000096,2000
2,GB,SFA000036,1000
3,GB,SFA000033,500
4,GB,SFA000033,1000


In [10]:
def fitter(X_train_enc,y_train):
    lm.fit(X_train_enc,y_train)
    return lm

def processing(df):
    y = df.iloc[:,-1]
    X = df.iloc[:,:-1]

    #Fit on train data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
    encoder = ce.OneHotEncoder(use_cat_names=True)
    X_train_enc = encoder.fit_transform(X_train)    
    lm = fitter(X_train_enc,y_train)
        
    predictions_train=lm.predict(X_train_enc)
    train_error = np.square(predictions_train - y_train)

    # here may be some metrics: 
    # get_stats(y_train, predictions_train)
  
    #Fit on all df data
    X_enc = encoder.fit_transform(X)
    lm = fitter(X_enc,y)
    all_predictions = lm.predict(X_enc)
    all_predictions = pd.DataFrame(all_predictions, columns=['DaysOfWork'])
    df = X.join(all_predictions['DaysOfWork'])
    return df

In [11]:
df_empl_predict = processing(df_empl_hist)

In [12]:
df_jr_predict = processing(df_vac_hist)

In [13]:
df_jr_predict.head(25)

,Country,Subfamily,DaysOfWork
0,FR,SFA000065,300.0
1,GB,SFA000096,200.0
2,GB,SFA000036,100.0
3,GB,SFA000086,400.0
4,GB,SFA000033,356.0


In [14]:
df_empl_predict.head(25)

,Country,Subfamily,DaysOfWork
0,FR,SFA000065,3000.0
1,GB,SFA000096,2000.0
2,GB,SFA000036,1000.0
3,GB,SFA000033,750.0
4,GB,SFA000033,750.0


In [15]:
def append_current_vac(predicts_empl, predicts_jr):
    predicts_empl = predicts_empl.rename(columns={"WorkStartDate": "WorkStartDate_work"})
    predicts_jr = predicts_jr.rename(columns={"WorkStartDate": "WorkStartDate_search"})
    result = predicts_empl.append(predicts_jr, ignore_index=True)
    result = result.drop_duplicates(subset="Position ID")
    return result

def apply_predicts_to_df_with_empl(empl_with_vac, 
                                   df_jr_predict, 
                                   df_empl_predict, 
                                   avghiretime=75, 
                                   avglifetime=850, 
                                   features_to_join=['Subfamily','Country']):
    
    """
    Apply intervals with predictors to arrays of real employees and vacancies (empl_with_vac)
    and returns the result megaarray at intervals
    
    """
    
    search_time = df_jr_predict.drop_duplicates()
    workee_time = df_empl_predict.drop_duplicates()
 
    result = pd.merge(empl_with_vac, workee_time, how='left', left_on=features_to_join, right_on=features_to_join)
    result = result.rename(columns={"DaysOfWork": "DaysOfWork_work"})
    result = pd.merge(result, search_time, how='left', left_on=['Subfamily','Country'], right_on=['Subfamily','Country'])
    result = result.rename(columns={"DaysOfWork": "DaysOfWork_search"})
    
    # antibug checks
    result.loc[result.DaysOfWork_search < 0, 'DaysOfWork_search'] = 0
    result.loc[result.DaysOfWork_work < 0, 'DaysOfWork_work'] = 0
    
    #EndDate 1  
    result['EstimateEndDate'] = pd.DatetimeIndex(result['WorkStartDate_work']) + 1*(pd.to_timedelta(result['DaysOfWork_work'].fillna(avglifetime).astype(int),'D'))
    
    # The problem of those who, according to the expectations of the algorithm, should have quit, but still work.
    # Let's replace them with the expected date of dismissal for the distant future    
    result.loc[result.EstimateEndDate < today, 'EstimateEndDate'] = pd.Timestamp('2030-01-01')
    
    #StartDate 2 and EndDate 2  
    # for employees
    result['EstimateNextHireDate'] = pd.DatetimeIndex(result['EstimateEndDate']) + 1*(pd.to_timedelta(result['DaysOfWork_search'].fillna(avglifetime).astype(int),'D'))
    # for vacancies
    result.loc[result['EstimateNextHireDate'].isnull(), 'EstimateNextHireDate'] = pd.DatetimeIndex(result['WorkStartDate_search']) + 1*(pd.to_timedelta(result['DaysOfWork_search'].fillna(avghiretime).astype(int),'D'))
    
    result.loc[result.EstimateNextHireDate < today, 'EstimateNextHireDate'] = today
    
    result['EstimateNextEndDate'] = pd.DatetimeIndex(result['EstimateNextHireDate']) + 1*(pd.to_timedelta(result['DaysOfWork_work'].fillna(avglifetime).astype(int),'D'))
    
    #StartDate 3 and EndDate 3
    result['EstimateNextNextHireDate'] = pd.DatetimeIndex(result['EstimateNextEndDate']) + 1*(pd.to_timedelta(result['DaysOfWork_search'].fillna(avglifetime).astype(int),'D'))
    result['EstimateNextNextEndDate'] = pd.DatetimeIndex(result['EstimateNextNextHireDate']) + 1*(pd.to_timedelta(result['DaysOfWork_work'].fillna(avglifetime).astype(int),'D'))
    
    result['UserID'] = result['UserID'].fillna(value=0).astype('int64')
    
    return result


In [16]:
empl_with_vac = append_current_vac(df_empl, df_vac)

In [17]:
empl_with_vac.head(25)

,UserID,Position ID,Country,Subfamily,Department,WorkStartDate_work,WorkStartDate_search
0,11391,POS012903,FR,SFA000065,FT,2019-05-17,NaN
1,7642,POS040759,GB,SFA000096,RnD,2019-08-11,NaN
2,9018,POS044017,GB,SFA000036,IT,2020-12-12,NaN
3,1359,POS027569,GB,SFA000086,RnD,2018-04-18,NaN
4,9552,POS040784,GB,SFA000033,IT,2021-04-20,NaN
5,NaN,POS041296,GB,SFA000065,FT,NaN,2021-05-17
6,NaN,POS027619,GB,SFA000096,RnD,NaN,2021-08-11
7,NaN,POS023292,GB,SFA000036,IT,NaN,2020-12-12
8,NaN,POS018157,GB,SFA000086,RnD,NaN,2021-04-18
9,NaN,POS009096,GB,SFA000033,IT,NaN,2021-04-20


In [18]:
predicts = apply_predicts_to_df_with_empl(empl_with_vac, df_jr_predict, df_empl_predict)

print(len(predicts))
predicts.head(25)

10


,UserID,Position ID,Country,Subfamily,Department,WorkStartDate_work,WorkStartDate_search,DaysOfWork_work,DaysOfWork_search,EstimateEndDate,EstimateNextHireDate,EstimateNextEndDate,EstimateNextNextHireDate,EstimateNextNextEndDate
0,11391,POS012903,FR,SFA000065,FT,2019-05-17,NaN,3000.0,300.0,2027-08-03,2028-05-29,2036-08-15,2037-06-11,2045-08-28
1,7642,POS040759,GB,SFA000096,RnD,2019-08-11,NaN,2000.0,200.0,2025-01-31,2025-08-18,2031-02-08,2031-08-26,2037-02-15
2,9018,POS044017,GB,SFA000036,IT,2020-12-12,NaN,1000.0,100.0,2023-09-07,2023-12-16,2026-09-10,2026-12-19,2029-09-13
3,1359,POS027569,GB,SFA000086,RnD,2018-04-18,NaN,NaN,400.0,2030-01-01,2031-02-05,2033-06-04,2034-07-09,2036-11-05
4,9552,POS040784,GB,SFA000033,IT,2021-04-20,NaN,750.0,356.0,2023-05-09,2024-04-29,2026-05-18,2027-05-09,2029-05-27
5,0,POS041296,GB,SFA000065,FT,NaN,2021-05-17,NaN,NaN,NaT,2021-09-30,2024-01-28,2026-05-27,2028-09-23
6,0,POS027619,GB,SFA000096,RnD,NaN,2021-08-11,2000.0,200.0,NaT,2022-02-26,2027-08-19,2028-03-05,2033-08-26
7,0,POS023292,GB,SFA000036,IT,NaN,2020-12-12,1000.0,100.0,NaT,2021-09-30,2024-06-25,2024-10-03,2027-06-29
8,0,POS018157,GB,SFA000086,RnD,NaN,2021-04-18,NaN,400.0,NaT,2022-05-23,2024-09-19,2025-10-24,2028-02-21
9,0,POS009096,GB,SFA000033,IT,NaN,2021-04-20,750.0,356.0,NaT,2022-04-11,2024-04-29,2025-04-20,2027-05-09


In the array above:

* UserID - employee's user ID (for vacancies = 0)
* Subfamily - subfamily in this position (approximately corresponds to the field of activity of the employee)
* WorkStartDate_work - start date of the employee in the organization
* WorkStartDate_search - date of opening the search for the position
* DaysOfWork_work - average employee work time for positions with these features
* DaysOfWork_search - average search time for a position with such features
* EstimateEndDate - the expected date the employee leaves the position
* EstimateNextHireDate - the expected date of hiring an employee for a position (for vacancies this is the initial hiring, for employees - hiring the next employee after the previous one is fired)
* EstimateNextEndDate - expected date of the next employee leaving the position

In [19]:
# predicts.to_csv('predicts.csv', sep=';', index=False)

In [20]:
def get_full_workmonth_of_positions(predicts):
    """
    Function to calculate the number of employees working in the organization as of the end of each month (by date intervals)
    Returns a pd-dataframe with the estimated number of employees on the date    
    
    """
    alldata = pd.DataFrame()
    # Iterate through two arrays (positions and calendar), compare with the date of the first prediction until it reaches the date of the prediction -- set to 1

    for i, position in predicts.iterrows():
        print('Progress: ', i, ' out of ', (len(predicts)))
        position_temp = position
        for i2, date in calenda.iterrows():
            date_temp = date
            if position['EstimateEndDate'] > date_temp[0]: #newrow[0] is a month in calenda
                alldata = alldata.append({'pos': position_temp['Position ID'], 'currentperiod': date_temp[0], 'isworking': 1, 'department': position_temp['Department']}, ignore_index=True)  
                
            # When the predicted date is reached -- start comparing with the date of the next hire and set it to 0    
            elif position['EstimateEndDate'] < date_temp[0] and position['EstimateNextHireDate'] > date_temp[0]:
                alldata = alldata.append({'pos': position_temp['Position ID'], 'currentperiod': date_temp[0], 'isworking': 0, 'department': position_temp['Department']}, ignore_index=True)
            elif position['EstimateNextHireDate'] < date_temp[0] and position['EstimateNextEndDate'] > date_temp[0]:
                alldata = alldata.append({'pos': position_temp['Position ID'], 'currentperiod': date_temp[0], 'isworking': 1, 'department': position_temp['Department']}, ignore_index=True)
            elif position['EstimateNextEndDate'] < date_temp[0] and position['EstimateNextNextHireDate'] > date_temp[0]:
                alldata = alldata.append({'pos': position_temp['Position ID'], 'currentperiod': date_temp[0], 'isworking': 0, 'department': position_temp['Department']}, ignore_index=True)
            elif position['EstimateNextNextHireDate'] < date_temp[0] and position['EstimateNextNextEndDate'] > date_temp[0]:
                alldata = alldata.append({'pos': position_temp['Position ID'], 'currentperiod': date_temp[0], 'isworking': 1, 'department': position_temp['Department']}, ignore_index=True)
            elif position['EstimateNextNextEndDate'] < date_temp[0]:
                alldata = alldata.append({'pos': position_temp['Position ID'], 'currentperiod': date_temp[0], 'isworking': 0, 'department': position_temp['Department']}, ignore_index=True)            
            else:
                pass
        resultmonth = alldata.groupby(by=['currentperiod','department'])['isworking'].agg(['sum']).reset_index()
    return resultmonth

general_headcount_by_month = get_full_workmonth_of_positions(predicts)

Progress:  0  out of  10
Progress:  1  out of  10
Progress:  2  out of  10
Progress:  3  out of  10
Progress:  4  out of  10
Progress:  5  out of  10
Progress:  6  out of  10
Progress:  7  out of  10
Progress:  8  out of  10
Progress:  9  out of  10


In [21]:
general_headcount_by_month.tail(50)

,currentperiod,department,sum
100,2024-03-31,IT,3.0
101,2024-03-31,RnD,4.0
102,2024-04-30,FT,1.0
103,2024-04-30,IT,3.0
104,2024-04-30,RnD,4.0
105,2024-05-31,FT,1.0
106,2024-05-31,IT,3.0
107,2024-05-31,RnD,4.0
108,2024-06-30,FT,1.0
109,2024-06-30,IT,2.0
